In [4]:
import yfinance as yf
import talib
import pandas as pd
import numpy as np
import vectorbt as vbt 
from vectorbt.portfolio.enums import OrderSide

# data preprocess
target_stock = "3037"
threshold = 0.05
stock_df = pd.read_csv(r'C:\Users\Admin\Desktop\量化交易\lstm 預測股價(squeeze中包含的技術分析)\LSTM_stock_predicting\data\3037 2023.csv', encoding='utf-8', encoding_errors= 'ignore')
five_day_min_low = talib.MIN(stock_df['Low'], timeperiod=5) # 五天最低價當作long strategy的停損點
five_day_max_high = talib.MAX(stock_df['High'], timeperiod=5) # 五天最高價當作short strategy的停損點
stock_df['five_day_min_low'] = five_day_min_low
stock_df['five_day_max_high'] = five_day_max_high
stock_df = stock_df[25:-1]
predicted_price_df = pd.read_csv(r"C:\Users\Admin\Desktop\量化交易\lstm 預測股價(squeeze中包含的技術分析)\LSTM_stock_predicting\predicted_result\LSTMModel 3037 2023 predicted prices.csv", encoding='utf-8', encoding_errors = 'ignore')
stock_df['Date'] = pd.to_datetime(stock_df['Date'])
stock_df.index = stock_df['Date']
stock_df['predicted_price'] = predicted_price_df['predicted_price'].values
stock_df['true_price'] = predicted_price_df['true_price'].values

# long short condition
stock_df['predicted_price_up'] = (stock_df['predicted_price'] > stock_df['predicted_price'].shift(1)).astype(int)
stock_df['actual_price_up'] = (stock_df['Close'] > stock_df['Close'].shift(1)).astype(int)
stock_df['long_condition'] = (stock_df['true_price'].shift(1)<=stock_df['predicted_price']) & ((stock_df['predicted_price']-stock_df['true_price'].shift(1))/stock_df['true_price'].shift(1) >= threshold)
stock_df['short_condition'] = (stock_df['true_price'].shift(1)>stock_df['predicted_price']) & ((stock_df['true_price'].shift(1)-stock_df['predicted_price'])/stock_df['predicted_price'] >= threshold)
stock_df['long_condition'] = stock_df['long_condition'].astype(int)
stock_df['short_condition'] = stock_df['short_condition'].astype(int)

stock_df

,Date,Capacity,Turnover,Open,High,Low,Close,Change,Transcation,buy_Dealer_Hedging,...,sell_Foreign_Investor,sell_Investment_Trust,five_day_min_low,five_day_max_high,predicted_price,true_price,predicted_price_up,actual_price_up,long_condition,short_condition
Date,,,,,,,,,,,,,,,,,,,,,
2023-02-17,2023-02-17,19483745,2654980784,136.5,138.0,135.0,136.0,-2.5,11572,88000,...,9866224,31967,128.0,140.5,0.467703,0.513502,0,0,0,0
2023-02-20,2023-02-20,11724345,1606472445,137.0,138.5,136.0,136.5,0.5,7882,111000,...,5119669,145000,130.0,140.5,0.471389,0.515564,1,1,0,1
2023-02-21,2023-02-21,14877279,2035759673,137.0,138.5,135.0,137.5,1.0,9987,174376,...,5270900,1112000,130.5,140.5,0.475025,0.519687,1,1,0,1
2023-02-22,2023-02-22,27076853,3636705253,135.0,136.0,133.5,133.5,-4.0,18518,135000,...,14715463,44692,133.5,140.5,0.478119,0.503195,1,0,0,1
2023-02-23,2023-02-23,25471038,3413583017,133.0,135.5,133.0,134.0,0.5,15533,231948,...,11134695,84000,133.0,138.5,0.479156,0.505257,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,2023-12-22,9369791,1653095057,178.0,179.0,175.0,175.0,-4.0,9894,34000,...,3996971,1845000,170.5,182.5,0.614134,0.674294,0,0,0,1
2023-12-25,2023-12-25,4296239,756810985,175.5,177.5,174.5,175.0,0.0,2824,93000,...,1678000,29637,170.5,180.0,0.612566,0.674294,0,0,0,1
2023-12-26,2023-12-26,7210903,1279503842,176.0,178.5,175.0,177.5,2.5,4458,58849,...,2289024,137637,170.5,180.0,0.611569,0.684601,0,1,0,1


In [5]:
# long strategy
signals_long = np.zeros(len(stock_df))
partition_size_long = 0
stop_loss_long = 0
for i in range(len(stock_df)):
    if partition_size_long == 0:
        if stock_df['long_condition'][i] == 1:
            signals_long[i] = 1
            partition_size_long = 1
            stop_loss_long = stock_df['five_day_min_low'][i]
    if partition_size_long == 1:
        if (stock_df['Close'][i] <= stop_loss_long) or (stock_df['short_condition'][i] == 1): # 停利停損
            signals_long[i] = -1
            stop_loss_long = 0 # 停損點重置
            partition_size_long = 0 # 平倉

# short strategy
signals_short = np.zeros(len(stock_df))
partition_size_short = 0
stop_loss_short = 0
for i in range(len(stock_df)):
    if partition_size_short == 0:
        if stock_df['short_condition'][i] == 1:
            signals_short[i] = -1
            partition_size_short = -1
            stop_loss_short = stock_df['five_day_max_high'][i]
    if partition_size_short == -1:
        if (stock_df['Close'][i] >= stop_loss_short) or (stock_df['long_condition'][i] == 1):# 停利停損
            signals_short[i] = 1
            stop_loss_short = 0 # 停損點重置
            partition_size_short = 0 # 平倉

entries_long = signals_long == 1
exits_long = signals_long == -1
entries_short = signals_short == -1
exits_short = signals_short == 1
pf = vbt.Portfolio.from_signals(stock_df['Open'], 
                                    entries=entries_long,
                                    exits=exits_long, 
                                    short_entries=entries_short,
                                    short_exits=exits_short,
                                    fees = 0,
                                    freq='1D',
                                    direction='both',)
# print(pf_long.stats().to_string()) # to_string()可以將全部結果攤開
print(pf.stats().to_string())
'''entries_short = signals_short == -1
exits_short = signals_short == 1
pf_short = vbt.Portfolio.from_signals(data['Close'], entries_short, exits_short, direction='shortonly')'''
# print(pf_short.stats().to_string())

Start                         2023-02-17 00:00:00
End                           2023-12-28 00:00:00
Period                          213 days 00:00:00
Start Value                                 100.0
End Value                              109.649171
Total Return [%]                         9.649171
Benchmark Return [%]                    30.769231
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                        14.392523
Max Drawdown Duration            71 days 00:00:00
Total Trades                                    6
Total Closed Trades                             5
Total Open Trades                               1
Open Trade PnL                          12.913714
Win Rate [%]                                 40.0
Best Trade [%]                           3.157895
Worst Trade [%]                         -3.244838
Avg Winning Trade [%]                    2.082303
Avg Losing Trade [%]                    -2.444451


C:\Users\Admin\AppData\Local\Temp\ipykernel_22608\2404176544.py:7: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\Admin\AppData\Local\Temp\ipykernel_22608\2404176544.py:23: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\Admin\AppData\Local\Temp\ipykernel_22608\2404176544.py:26: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\Admin\AppData\Local\Temp\ipykernel_22608\2404176544.py:28: FutureWarning:

Series.__ge

"entries_short = signals_short == -1\nexits_short = signals_short == 1\npf_short = vbt.Portfolio.from_signals(data['Close'], entries_short, exits_short, direction='shortonly')"

In [6]:
pf.plot().show()